In [1]:
# %load jupyter_default.py
import pandas as pd
import numpy as np
import os
import re
import datetime
import time
import glob
import json
from tqdm import tqdm_notebook
from colorama import Fore, Style

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn as sns

%config InlineBackend.figure_format='retina'
sns.set() # Revert to matplotlib defaults
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['axes.labelpad'] = 20
plt.rcParams['legend.fancybox'] = True
plt.style.use('ggplot')

SMALL_SIZE, MEDIUM_SIZE, BIGGER_SIZE = 14, 16, 20
plt.rc('font', size=SMALL_SIZE)
plt.rc('axes', titlesize=SMALL_SIZE)
plt.rc('axes', labelsize=MEDIUM_SIZE)
plt.rc('xtick', labelsize=SMALL_SIZE)
plt.rc('ytick', labelsize=SMALL_SIZE)
plt.rc('legend', fontsize=MEDIUM_SIZE)
plt.rc('axes', titlesize=BIGGER_SIZE)

def savefig(plt, name):
    plt.savefig(f'../../figures/{name}.png', bbox_inches='tight', dpi=300)

%reload_ext autoreload
%autoreload 2
    
%reload_ext version_information
%version_information pandas, numpy

Software versions
Python 3.6.8 64bit [GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
IPython 6.4.0
OS Darwin 16.7.0 x86_64 i386 64bit
pandas 0.23.4
numpy 1.14.2
Mon Apr 29 10:54:52 2019 PDT

In [2]:
from google.cloud import bigquery

In [3]:
from dotenv import load_dotenv
load_dotenv('../../.env')

True

In [4]:
client = bigquery.Client()

# GA Data Mining
Alex's development notebook for miscellaneous rough work.

## Misc

Note:
> The first 1 TB of query data processed per month is free

I've downloaded it all as JSON, but I'm going to use bigquery SQL until I run out of calls

In [5]:
query_job = client.query("""
    SELECT
        *
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
    LIMIT 10
""")

results = query_job.result()  # Waits for job to complete.

In [6]:
data = [r for r in results]

Here is one row of data

In [7]:
dict(data[0].items())

{'visitorId': None,
 'visitNumber': 1,
 'visitId': 1501583974,
 'visitStartTime': 1501583974,
 'date': '20170801',
 'totals': {'visits': 1,
  'hits': 1,
  'pageviews': 1,
  'timeOnSite': None,
  'bounces': 1,
  'transactions': None,
  'transactionRevenue': None,
  'newVisits': 1,
  'screenviews': None,
  'uniqueScreenviews': None,
  'timeOnScreen': None,
  'totalTransactionRevenue': None,
  'sessionQualityDim': 1},
 'trafficSource': {'referralPath': None,
  'campaign': '(not set)',
  'source': '(direct)',
  'medium': '(none)',
  'keyword': None,
  'adContent': None,
  'adwordsClickInfo': {'campaignId': None,
   'adGroupId': None,
   'creativeId': None,
   'criteriaId': None,
   'page': None,
   'slot': None,
   'criteriaParameters': 'not available in demo dataset',
   'gclId': None,
   'customerId': None,
   'adNetworkType': None,
   'targetingCriteria': None,
   'isVideoAd': None},
  'isTrueDirect': None,
  'campaignCode': None},
 'device': {'browser': 'Chrome',
  'browserVersion': 'n

In [12]:
json.dumps(dict(data[0].items()))

'{"visitorId": null, "visitNumber": 1, "visitId": 1501583974, "visitStartTime": 1501583974, "date": "20170801", "totals": {"visits": 1, "hits": 1, "pageviews": 1, "timeOnSite": null, "bounces": 1, "transactions": null, "transactionRevenue": null, "newVisits": 1, "screenviews": null, "uniqueScreenviews": null, "timeOnScreen": null, "totalTransactionRevenue": null, "sessionQualityDim": 1}, "trafficSource": {"referralPath": null, "campaign": "(not set)", "source": "(direct)", "medium": "(none)", "keyword": null, "adContent": null, "adwordsClickInfo": {"campaignId": null, "adGroupId": null, "creativeId": null, "criteriaId": null, "page": null, "slot": null, "criteriaParameters": "not available in demo dataset", "gclId": null, "customerId": null, "adNetworkType": null, "targetingCriteria": null, "isVideoAd": null}, "isTrueDirect": null, "campaignCode": null}, "device": {"browser": "Chrome", "browserVersion": "not available in demo dataset", "browserSize": "not available in demo dataset", "o

We got 10 total rows

Let's get some info about the dataset

In [34]:
dataset_id = 'bigquery-public-data.google_analytics_sample'
dataset = client.get_dataset(dataset_id)

In [35]:
full_dataset_id = "{}.{}".format(dataset.project, dataset.dataset_id)
friendly_name = dataset.friendly_name
print(
    "Got dataset '{}' with friendly_name '{}'.".format(
        full_dataset_id, friendly_name
    )
)

# View dataset properties
print("Description: {}".format(dataset.description))
print("Labels:")
labels = dataset.labels
if labels:
    for label, value in labels.items():
        print("\t{}: {}".format(label, value))
else:
    print("\tDataset has no labels defined.")

# View tables in dataset
print("Tables:")
tables = list(client.list_tables(dataset))  # API request(s)
if tables:
    for table in tables:
        print("\t{}".format(table.table_id))
else:
    print("\tThis dataset does not contain any tables.")


Got dataset 'bigquery-public-data.google_analytics_sample' with friendly_name 'None'.
Description: None
Labels:
	Dataset has no labels defined.
Tables:
	ga_sessions_20160801
	ga_sessions_20160802
	ga_sessions_20160803
	ga_sessions_20160804
	ga_sessions_20160805
	ga_sessions_20160806
	ga_sessions_20160807
	ga_sessions_20160808
	ga_sessions_20160809
	ga_sessions_20160810
	ga_sessions_20160811
	ga_sessions_20160812
	ga_sessions_20160813
	ga_sessions_20160814
	ga_sessions_20160815
	ga_sessions_20160816
	ga_sessions_20160817
	ga_sessions_20160818
	ga_sessions_20160819
	ga_sessions_20160820
	ga_sessions_20160821
	ga_sessions_20160822
	ga_sessions_20160823
	ga_sessions_20160824
	ga_sessions_20160825
	ga_sessions_20160826
	ga_sessions_20160827
	ga_sessions_20160828
	ga_sessions_20160829
	ga_sessions_20160830
	ga_sessions_20160831
	ga_sessions_20160901
	ga_sessions_20160902
	ga_sessions_20160903
	ga_sessions_20160904
	ga_sessions_20160905
	ga_sessions_20160906
	ga_sessions_20160907
	ga_sessions

Export the data

In [11]:
def dump_jsonl(big_query_results, f_path):
    """
    Feed in big query result, e.g.
    
    >>> query_job = client.query('''
    >>>     SELECT *
    >>>     FROM `bigquery-public-data.google_analytics_sample.{}`
    >>>     LIMIT 10
    >>> '''.format(table_id))
    >>> results = query_job.result()  # Waits for job to complete.
    >>> dump_jsonl(results, '../../data/raw/{}.jsonl'.format(table_id))
    
    """
    print('Writing {} lines to {}'.format(len(big_query_results), f_path))
    with open(f_path, 'w') as f:
        for result in results:
            f.write('{}\n'.format(
                json.dumps(dict(big_query_results.items()))
            ))

## Predicting Product Revenue

### Read from bigquery

In [14]:
table_id = 'ga_sessions_20160801'
query_job = client.query('''
    select date, sum(totals.totalTransactionRevenue)
    from `bigquery-public-data.google_analytics_sample.{}`
    group by date
    limit 10;
'''.format(table_id))
results = query_job.result()

In [18]:
dict(results)

{'20160801': 6288060000}

In [49]:
%%time
"""
Using bigquery
"""

def pull_daily_data(verbose=False):
    dataset = client.get_dataset('bigquery-public-data.google_analytics_sample')

    data = []
    for table in tqdm_notebook(list(client.list_tables(dataset))):
        if verbose:
            print('Querying {}'.format(table.table_id))
        query_job = client.query('''
            select
              date,
              sum(totals.visits),
              sum(totals.pageviews),
              sum(totals.transactions),
              sum(totals.transactionRevenue)
            from `bigquery-public-data.google_analytics_sample.{}`
            group by date;
        '''.format(table.table_id))
        results = query_job.result().to_dataframe()
        results.columns = ['date', 'visits', 'pageviews', 'transactions', 'transactionRevenue']
        data.append(results)

    df = pd.concat(data, ignore_index=True, sort=False)
    return df

bq_results = pull_daily_data()


CPU times: user 7.94 s, sys: 856 ms, total: 8.8 s
Wall time: 11min 29s


In [53]:
bq_results.head()

,date,visits,pageviews,transactions,transactionRevenue
0,20160801,1711,9843,34,6116060000
1,20160802,2140,11784,18,1361190000
2,20160803,2890,13724,None,None
3,20160804,3161,13326,17,1182890000
4,20160805,2702,13585,42,5594260000


### Read `jsonl` from local

In [50]:
%%time
"""
Using local jsonl
"""
ERRORS = []

def pull_daily_data(verbose=False, raise_errors=False):
    dataset = sorted(glob.glob('../../data/raw/*.jsonl'))

    data = []
    for table in tqdm_notebook(dataset):
        if verbose:
            print('Scanning {}'.format(table))
        with open(table, 'r') as f:
            table_data = []
            for line in f:
                d = json.loads(line)
                date = d['date']
                d = d['totals']
                try:
                    table_data.append([
                        date,
                        d['visits'],
                        d['pageviews'],
                        d['transactions'],
                        d['transactionRevenue'],
                    ])
                except Exception as e:
                    if verbose:
                        print('Error raised when reading row:\n{}'.format(e))
                    ERRORS.append([table, e])
                    if raise_errors:
                        raise(e)

            cols = ['date', 'visits', 'pageviews', 'transactions', 'transactionRevenue']
            results = (
                pd.DataFrame(table_data, columns=cols)
                    .groupby('date')[['visits', 'pageviews', 'transactions', 'transactionRevenue']]
                    .sum().reset_index()
            )
            data.append(results)

    df = pd.concat(data, ignore_index=True, sort=False)
    return df

jsonl_results = pull_daily_data()


CPU times: user 5min 17s, sys: 19.2 s, total: 5min 36s
Wall time: 5min 50s


In [54]:
jsonl_results.head()

,date,visits,pageviews,transactions,transactionRevenue
0,20160801,1711,9843.0,34.0,6.116060e+09
1,20160802,2140,11784.0,18.0,1.361190e+09
2,20160803,2890,13724.0,NaN,NaN
3,20160804,3161,13326.0,17.0,1.182890e+09
4,20160805,2702,13585.0,42.0,5.594260e+09


### Read `jsonl` from google drive

In [7]:
%%time
"""
Using gdrive jsonl
"""
ERRORS = []

def pull_daily_data(verbose=False, raise_errors=False):
    dataset = sorted(glob.glob('/Volumes/GoogleDrive/My Drive/bigquery_ga_sample/*.jsonl'))

    data = []
    for table in tqdm_notebook(dataset):
        if verbose:
            print('Scanning {}'.format(table))
        with open(table, 'r') as f:
            table_data = []
            for line in f:
                d = json.loads(line)
                date = d['date']
                d = d['totals']
                try:
                    table_data.append([
                        date,
                        d['visits'],
                        d['pageviews'],
                        d['transactions'],
                        d['transactionRevenue'],
                    ])
                except Exception as e:
                    if verbose:
                        print('Error raised when reading row:\n{}'.format(e))
                    ERRORS.append([table, e])
                    if raise_errors:
                        raise(e)

            cols = ['date', 'visits', 'pageviews', 'transactions', 'transactionRevenue']
            results = (
                pd.DataFrame(table_data, columns=cols)
                    .groupby('date')[['visits', 'pageviews', 'transactions', 'transactionRevenue']]
                    .sum().reset_index()
            )
            data.append(results)

    df = pd.concat(data, ignore_index=True, sort=False)
    return df

jsonl_gdrive_results = pull_daily_data()


CPU times: user 7min 55s, sys: 1min 56s, total: 9min 52s
Wall time: 23min 14s


In [8]:
jsonl_gdrive_results

,date,visits,pageviews,transactions,transactionRevenue
0,20160801,1711,9843.0,34.0,6.116060e+09
1,20160802,2140,11784.0,18.0,1.361190e+09
2,20160803,2890,13724.0,NaN,NaN
3,20160804,3161,13326.0,17.0,1.182890e+09
4,20160805,2702,13585.0,42.0,5.594260e+09
5,20160806,1663,6926.0,10.0,1.891040e+09
6,20160807,1622,7239.0,16.0,2.410730e+09
7,20160808,2815,13179.0,36.0,5.273810e+09
8,20160809,2851,13411.0,44.0,5.596400e+09
9,20160810,2757,13003.0,46.0,4.284210e+09


In [7]:
from IPython.display import HTML
HTML('<style>div.text_cell_render{font-size:130%;padding-top:50px;padding-bottom:50px}</style>')